In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load dataset ===
df = pd.read_csv("compas_cleaned.csv")
df["two_year_recid"] = df["two_year_recid"].astype(int)

# === Define sensitive attribute ===
df["race"] = df["race_African-American"]  # Privileged group = 1

# === Train-Test Split ===
X = df.drop(columns=["two_year_recid"])
y = df["two_year_recid"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_test = y_test.reset_index(drop=True)
race_test = X_test["race"].reset_index(drop=True)

# === Model training function ===
def train_models(X_train, y_train, sample_weight=None):
    models = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(probability=True),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
    for name, model in models.items():
        model.fit(X_train, y_train, sample_weight=sample_weight)
    return models

# === Fairness Evaluation ===
def evaluate_fairness(models, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        group_priv = race_test == 1  # African-American
        group_unpriv = race_test == 0  # Caucasian

        err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
        err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
        erd = err_unpriv - err_priv

        def tpr(y_true, y_pred):
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            return tp / (tp + fn) if (tp + fn) > 0 else 0

        tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
        tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
        tprd = tpr_unpriv - tpr_priv

        abroca = np.nan
        if y_prob is not None:
            try:
                fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
                fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
                auc_priv = auc(fpr_priv, tpr_priv_vals)
                auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
                abroca = abs(auc_priv - auc_unpriv)
            except:
                pass

        fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")

        print(f"{name:<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")
        
# === 0. Absolute Fairness (No mitigation, sensitive attribute included) ===
models_abs = train_models(X_train, y_train)
evaluate_fairness(models_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression ===
X_train_sup = X_train.drop(columns=["race", "race_African-American", "race_Caucasian"])
X_test_sup = X_test.drop(columns=["race", "race_African-American", "race_Caucasian"])
models_sup = train_models(X_train_sup, y_train)
evaluate_fairness(models_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover ===
cr = CorrelationRemover(sensitive_feature_ids=["race"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
models_cr = train_models(X_train_cr, y_train)
evaluate_fairness(models_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover ===
df_train = pd.concat([X_train, y_train.rename("two_year_recid")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                               protected_attribute_names=["race"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
models_dir = train_models(X_train_dir, y_train)
evaluate_fairness(models_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing ===
dataset_rw = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                             protected_attribute_names=["race"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race": 1}], unprivileged_groups=[{"race": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
models_rw = train_models(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(models_rw, X_test, y_test, race_test, "Reweighing")


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.

Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0438     0.0099     -0.1311    0.9384    
Logistic Regression  0.0476     -0.0148    -0.2738    0.8879    
Random Forest        0.0416     -0.0024    -0.1654    0.9302    
SVM                  0.0514     0.0148     -0.3229    0.8703    
XGBoost              0.0701     0.0430     -0.2616    0.8751    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0423 

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load dataset ===
df = pd.read_csv("generated_data_CLLM_prompt_COMPAS.csv")
df["two_year_recid"] = df["two_year_recid"].astype(int)

# === Define sensitive attribute ===
df["race"] = df["race_African-American"]  # Privileged group = 1

# === Train-Test Split ===
X = df.drop(columns=["two_year_recid"])
y = df["two_year_recid"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_test = y_test.reset_index(drop=True)
race_test = X_test["race"].reset_index(drop=True)

# === Model training function ===
def train_models(X_train, y_train, sample_weight=None):
    models = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(probability=True),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
    for name, model in models.items():
        model.fit(X_train, y_train, sample_weight=sample_weight)
    return models

# === Fairness Evaluation ===
def evaluate_fairness(models, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        group_priv = race_test == 1  # African-American
        group_unpriv = race_test == 0  # Caucasian

        err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
        err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
        erd = err_unpriv - err_priv

        def tpr(y_true, y_pred):
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            return tp / (tp + fn) if (tp + fn) > 0 else 0

        tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
        tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
        tprd = tpr_unpriv - tpr_priv

        abroca = np.nan
        if y_prob is not None:
            try:
                fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
                fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
                auc_priv = auc(fpr_priv, tpr_priv_vals)
                auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
                abroca = abs(auc_priv - auc_unpriv)
            except:
                pass

        fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")

        print(f"{name:<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")
        
# === 0. Absolute Fairness (No mitigation, sensitive attribute included) ===
models_abs = train_models(X_train, y_train)
evaluate_fairness(models_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression ===
X_train_sup = X_train.drop(columns=["race", "race_African-American", "race_Caucasian"])
X_test_sup = X_test.drop(columns=["race", "race_African-American", "race_Caucasian"])
models_sup = train_models(X_train_sup, y_train)
evaluate_fairness(models_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover ===
cr = CorrelationRemover(sensitive_feature_ids=["race"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
models_cr = train_models(X_train_cr, y_train)
evaluate_fairness(models_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover ===
df_train = pd.concat([X_train, y_train.rename("two_year_recid")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                               protected_attribute_names=["race"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
models_dir = train_models(X_train_dir, y_train)
evaluate_fairness(models_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing ===
dataset_rw = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                             protected_attribute_names=["race"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race": 1}], unprivileged_groups=[{"race": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
models_rw = train_models(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(models_rw, X_test, y_test, race_test, "Reweighing")



Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0096     -0.0128    -0.0060    0.9905    
Logistic Regression  0.0259     -0.0099    -0.0168    0.9825    
Random Forest        0.0187     -0.0084    0.0187     0.9847    
SVM                  0.0281     -0.0247    0.0311     0.9721    
XGBoost              0.0110     -0.0024    0.0356     0.9837    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0212     -0.0234    -0.0137    0.9806    
Logistic Regression  0.0259     -0.0159    -0.0168    0.9805    
Random Forest        0.0175     -0.0205 

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load dataset ===
df = pd.read_csv("generated_data_Our_prompt_COMPAS.csv")
df["two_year_recid"] = df["two_year_recid"].astype(int)

# === Define sensitive attribute ===
df["race"] = df["race_African-American"]  # Privileged group = 1

# === Train-Test Split ===
X = df.drop(columns=["two_year_recid"])
y = df["two_year_recid"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_test = y_test.reset_index(drop=True)
race_test = X_test["race"].reset_index(drop=True)

# === Model training function ===
def train_models(X_train, y_train, sample_weight=None):
    models = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(probability=True),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
    for name, model in models.items():
        model.fit(X_train, y_train, sample_weight=sample_weight)
    return models

# === Fairness Evaluation ===
def evaluate_fairness(models, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        group_priv = race_test == 1  # African-American
        group_unpriv = race_test == 0  # Caucasian

        err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
        err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
        erd = err_unpriv - err_priv

        def tpr(y_true, y_pred):
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            return tp / (tp + fn) if (tp + fn) > 0 else 0

        tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
        tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
        tprd = tpr_unpriv - tpr_priv

        abroca = np.nan
        if y_prob is not None:
            try:
                fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
                fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
                auc_priv = auc(fpr_priv, tpr_priv_vals)
                auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
                abroca = abs(auc_priv - auc_unpriv)
            except:
                pass

        fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")

        print(f"{name:<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")
        
# === 0. Absolute Fairness (No mitigation, sensitive attribute included) ===
models_abs = train_models(X_train, y_train)
evaluate_fairness(models_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression ===
X_train_sup = X_train.drop(columns=["race", "race_African-American", "race_Caucasian"])
X_test_sup = X_test.drop(columns=["race", "race_African-American", "race_Caucasian"])
models_sup = train_models(X_train_sup, y_train)
evaluate_fairness(models_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover ===
cr = CorrelationRemover(sensitive_feature_ids=["race"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
models_cr = train_models(X_train_cr, y_train)
evaluate_fairness(models_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover ===
df_train = pd.concat([X_train, y_train.rename("two_year_recid")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                               protected_attribute_names=["race"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
models_dir = train_models(X_train_dir, y_train)
evaluate_fairness(models_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing ===
dataset_rw = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                             protected_attribute_names=["race"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race": 1}], unprivileged_groups=[{"race": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
models_rw = train_models(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(models_rw, X_test, y_test, race_test, "Reweighing")



Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0273     -0.0108    -0.0717    0.9634    
Logistic Regression  0.0337     -0.0263    0.0265     0.9712    
Random Forest        0.0241     0.0377     -0.0830    0.9517    
SVM                  0.0287     -0.0296    -0.0090    0.9776    
XGBoost              0.0430     -0.0040    -0.0929    0.9533    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0163     0.0061     -0.0929    0.9616    
Logistic Regression  0.0337     -0.0236    0.0009     0.9806    
Random Forest        0.0320     -0.0047 

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from fairlearn.preprocessing import CorrelationRemover
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.datasets import StandardDataset

# === Load dataset ===
df = pd.read_csv("compas_synthetic_data_1000_200_epochs.csv")
df["two_year_recid"] = df["two_year_recid"].astype(int)

# === Define sensitive attribute ===
df["race"] = df["race_African-American"]  # Privileged group = 1

# === Train-Test Split ===
X = df.drop(columns=["two_year_recid"])
y = df["two_year_recid"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_test = y_test.reset_index(drop=True)
race_test = X_test["race"].reset_index(drop=True)

# === Model training function ===
def train_models(X_train, y_train, sample_weight=None):
    models = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(probability=True),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
    }
    for name, model in models.items():
        model.fit(X_train, y_train, sample_weight=sample_weight)
    return models

# === Fairness Evaluation ===
def evaluate_fairness(models, X_test, y_test, race_test, title):
    print(f"\nFairness Metrics: {title}")
    print("--------------------------------------------------------------------------")
    print(f"{'Model':<20} {'ABROCA':<10} {'ERD':<10} {'TPRD':<10} {'Fairness':<10}")
    print("--------------------------------------------------------------------------")

    for name, model in models.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        group_priv = race_test == 1  # African-American
        group_unpriv = race_test == 0  # Caucasian

        err_priv = np.mean(y_pred[group_priv] != y_test[group_priv])
        err_unpriv = np.mean(y_pred[group_unpriv] != y_test[group_unpriv])
        erd = err_unpriv - err_priv

        def tpr(y_true, y_pred):
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            return tp / (tp + fn) if (tp + fn) > 0 else 0

        tpr_priv = tpr(y_test[group_priv], y_pred[group_priv])
        tpr_unpriv = tpr(y_test[group_unpriv], y_pred[group_unpriv])
        tprd = tpr_unpriv - tpr_priv

        abroca = np.nan
        if y_prob is not None:
            try:
                fpr_priv, tpr_priv_vals, _ = roc_curve(y_test[group_priv], y_prob[group_priv])
                fpr_unpriv, tpr_unpriv_vals, _ = roc_curve(y_test[group_unpriv], y_prob[group_unpriv])
                auc_priv = auc(fpr_priv, tpr_priv_vals)
                auc_unpriv = auc(fpr_unpriv, tpr_unpriv_vals)
                abroca = abs(auc_priv - auc_unpriv)
            except:
                pass

        fairness = (3 - abs(abroca) - abs(erd) - abs(tprd)) / 3 if not np.isnan(abroca) else float("nan")

        print(f"{name:<20} {abroca:<10.4f} {erd:<10.4f} {tprd:<10.4f} {fairness:<10.4f}")
        
# === 0. Absolute Fairness (No mitigation, sensitive attribute included) ===
models_abs = train_models(X_train, y_train)
evaluate_fairness(models_abs, X_test, y_test, race_test, "Absolute (No Mitigation)")

# === 1. Suppression ===
X_train_sup = X_train.drop(columns=["race", "race_African-American", "race_Caucasian"])
X_test_sup = X_test.drop(columns=["race", "race_African-American", "race_Caucasian"])
models_sup = train_models(X_train_sup, y_train)
evaluate_fairness(models_sup, X_test_sup, y_test, race_test, "Suppression")

# === 2. Correlation Remover ===
cr = CorrelationRemover(sensitive_feature_ids=["race"])
cr.fit(X_train)
X_train_cr = cr.transform(X_train)
X_test_cr = cr.transform(X_test)
models_cr = train_models(X_train_cr, y_train)
evaluate_fairness(models_cr, X_test_cr, y_test, race_test, "Correlation Remover")

# === 3. Disparate Impact Remover ===
df_train = pd.concat([X_train, y_train.rename("two_year_recid")], axis=1)
dataset_dir = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                               protected_attribute_names=["race"], privileged_classes=[[1]])
dir = DisparateImpactRemover(repair_level=1.0)
transformed = dir.fit_transform(dataset_dir)
X_train_dir = pd.DataFrame(transformed.features, columns=X_train.columns)
models_dir = train_models(X_train_dir, y_train)
evaluate_fairness(models_dir, X_test, y_test, race_test, "Disparate Impact Remover")

# === 4. Reweighing ===
dataset_rw = StandardDataset(df_train, label_name="two_year_recid", favorable_classes=[0],
                             protected_attribute_names=["race"], privileged_classes=[[1]])
rw = Reweighing(privileged_groups=[{"race": 1}], unprivileged_groups=[{"race": 0}])
transformed_rw = rw.fit_transform(dataset_rw)
X_train_rw = pd.DataFrame(transformed_rw.features, columns=X_train.columns)
models_rw = train_models(X_train_rw, y_train, sample_weight=transformed_rw.instance_weights)
evaluate_fairness(models_rw, X_test, y_test, race_test, "Reweighing")



Fairness Metrics: Absolute (No Mitigation)
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0195     -0.0017    -0.1222    0.9522    
Logistic Regression  0.0533     0.0027     -0.0153    0.9762    
Random Forest        0.0353     0.0584     -0.0745    0.9439    
SVM                  0.0648     -0.0624    0.0805     0.9308    
XGBoost              0.0612     0.0710     -0.0885    0.9264    

Fairness Metrics: Suppression
--------------------------------------------------------------------------
Model                ABROCA     ERD        TPRD       Fairness  
--------------------------------------------------------------------------
Decision Tree        0.0555     0.0571     -0.1026    0.9283    
Logistic Regression  0.0537     0.0027     -0.0268    0.9723    
Random Forest        0.0130     -0.0221 